<a href="https://colab.research.google.com/github/veronica1908/MARKETING_E2_A_III/blob/main/b_exploracion_limpieza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BIBLIOTECAS Y LIBRERÍAS

In [118]:
# Instalar bibliotecas necesarias
!pip install sqlalchemy
!pip install mysql-connector-python

In [119]:
# Importar bibliotecas
import pandas as pd
import sqlite3
import plotly.graph_objects as go
import plotly.express as px
from sqlalchemy import create_engine
import urllib.request

#VERIFICACIÓN DE LA BASE DE DATOS EN SQL

##Ver las tablas de la base de datos

In [120]:
# Conectarse a la base de datos
conn = sqlite3.connect('db_movies')

# Crear un cursor
cur = conn.cursor()

# Obtener una lista de tablas en la base de datos
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()

# Imprimir los nombres de las tablas para verificar la información
for table in tables:
    print(table[0])


ratings
movies


##Ver las primeras filas de las tablas

In [121]:
# Leer las primeras filas de la tabla 'ratings'
df_ratings = pd.read_sql_query("SELECT * FROM ratings LIMIT 5;", conn)
print("Primeras filas de la tabla 'ratings':")
print(df_ratings)

# Leer las primeras filas de la tabla 'movies'
df_movies = pd.read_sql_query("SELECT * FROM movies LIMIT 5;", conn)
print("\nPrimeras filas de la tabla 'movies':")
print(df_movies)


Primeras filas de la tabla 'ratings':
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931

Primeras filas de la tabla 'movies':
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


##Ver el tipo de datos de cada tabla

In [122]:
# Obtener información sobre las columnas de la tabla 'ratings'
cur.execute("PRAGMA table_info(ratings)")
columns_ratings = cur.fetchall()

# Imprimir el nombre de la columna y su tipo de datos para la tabla 'ratings'
print("Tipos de datos de las columnas en la tabla 'ratings':")
for column in columns_ratings:
    print(column[1] + ": " + column[2])

# Obtener información sobre las columnas de la tabla 'movies'
cur.execute("PRAGMA table_info(movies)")
columns_movies = cur.fetchall()

# Imprimir el nombre de la columna y su tipo de datos para la tabla 'movies'
print("\nTipos de datos de las columnas en la tabla 'movies':")
for column in columns_movies:
    print(column[1] + ": " + column[2])

Tipos de datos de las columnas en la tabla 'ratings':
userId: INTEGER
movieId: INTEGER
rating: REAL
timestamp: INTEGER

Tipos de datos de las columnas en la tabla 'movies':
movieId: INTEGER
title: TEXT
genres: TEXT


#TRAER TABLAS A PYTHON

##Tabla ratings

In [123]:
# Consulta SQL para seleccionar todos los datos de la tabla especificada
query = f"SELECT * FROM {'ratings'}"

# Leer los datos de la tabla en un DataFrame de Pandas
dfr = pd.read_sql_query(query, conn)

# Mostrar las primeras filas del DataFrame
print("Primeras filas de la tabla '{}':".format('ratings'))
print(dfr.head())

Primeras filas de la tabla 'ratings':
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


##Tabla movies

In [124]:
# Consulta SQL para seleccionar todos los datos de la tabla especificada
query = f"SELECT * FROM {'movies'}"

# Leer los datos de la tabla en un DataFrame de Pandas
dfm = pd.read_sql_query(query, conn)

# Mostrar las primeras filas del DataFrame
print("Primeras filas de la tabla '{}':".format('movies'))
print(dfm.head())

Primeras filas de la tabla 'movies':
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


#EXPLORACIÓN INICIAL

In [125]:
# Identificar los campos de cruce
# Suponiendo que 'movieId' es el campo de cruce en ambas tablas
print("\nCampos de cruce:")
print("Tabla 'ratings':", dfr.columns)
print("Tabla 'movies':", dfm.columns)

# Verificar que los campos de cruce estén en el mismo formato
# Comprobación del formato de 'movieId' en ambas tablas
print("\nFormato de 'movieId':")
print("Tabla 'ratings':", dfr['movieId'].dtype)
print("Tabla 'movies':", dfm['movieId'].dtype)

# Verificar duplicados en ambas tablas
duplicates_ratings = dfr.duplicated().sum()
duplicates_movies = dfm.duplicated().sum()

print("\nCantidad de duplicados en la tabla 'ratings':", duplicates_ratings)
print("Cantidad de duplicados en la tabla 'movies':", duplicates_movies)


Campos de cruce:
Tabla 'ratings': Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')
Tabla 'movies': Index(['movieId', 'title', 'genres'], dtype='object')

Formato de 'movieId':
Tabla 'ratings': int64
Tabla 'movies': int64

Cantidad de duplicados en la tabla 'ratings': 0
Cantidad de duplicados en la tabla 'movies': 0


##Descripción base ratings

##Distribución de las calificaciones

In [126]:
# Consulta SQL para calcular la distribución de calificaciones
query = """
        SELECT
            rating,
            COUNT(*) AS conteo
        FROM
            ratings
        GROUP BY
            rating
        ORDER BY
            conteo DESC
        """

# Ejecutar la consulta y cargar los resultados en un DataFrame
dfr2 = pd.read_sql_query(query, conn)

# Imprimir el DataFrame resultante
print(dfr2)

   rating  conteo
0     4.0   26818
1     3.0   20047
2     5.0   13211
3     3.5   13136
4     4.5    8551
5     2.0    7551
6     2.5    5550
7     1.0    2811
8     1.5    1791
9     0.5    1370


In [127]:
# Crear el objeto de datos de la gráfica de barras
data = go.Bar(
    x=dfr2.rating,
    y=dfr2.conteo,
    text=dfr2.conteo,
    textposition="outside"
)

# Crear el diseño de la gráfica
layout = go.Layout(
    title="Conteo de calificaciones",
    xaxis={'title': 'Rating'},
    yaxis={'title': 'Count'}
)

# Crear la figura con los datos y el diseño
fig = go.Figure(data=data, layout=layout)

# Mostrar la figura
fig.show()

El rating 4.0 es el que tiene mayor cantidad de calificaciones con 26818 y el que menos calificaciones tiene es el rating 0.5 con 1370 calificaciones.

##Cantidad de calificaciones por usuario

In [128]:
# Calcular cuántas películas calificó cada usuario
rating_users = pd.read_sql('''SELECT userId as user_id,
                                     COUNT(*) as cnt_rat
                              FROM ratings
                              GROUP BY userId
                              ORDER BY cnt_rat DESC''', conn)

# Imprimir el DataFrame
print(rating_users)

     user_id  cnt_rat
0        414     2698
1        599     2478
2        474     2108
3        448     1864
4        274     1346
..       ...      ...
605      207       20
606      194       20
607      189       20
608      147       20
609       53       20

[610 rows x 2 columns]


El usuario 414 es el que ha realizado mayor cantidad de calificaciones con un total de 2698 películas. El usuario 53 es el que menos películas ha calificado, con un total de 20.

In [129]:
# Crear un histograma de la frecuencia del número de calificaciones por usuario
fig = px.histogram(rating_users, x='cnt_rat', title='Histograma de la frecuencia del número de calificaciones por usuario')
fig.show()

In [130]:
rating_users.describe()

,user_id,cnt_rat
count,610.000000,610.000000
mean,305.500000,165.304918
std,176.236111,269.480584
min,1.000000,20.000000
25%,153.250000,35.000000
50%,305.500000,70.500000
75%,457.750000,168.000000
max,610.000000,2698.000000


**Cantidad promedio de calificaciones por usuario:** La cantidad promedio de calificaciones por usuario es de aproximadamente 165 películas.

**Número mínimo y máximo de calificaciones por usuario:** El número mínimo de calificaciones por usuario es 20, lo que indica que hay usuarios que han calificado relativamente pocas películas. El número máximo de calificaciones por usuario es 2698, lo que indica que hay usuarios que han calificado un gran número de películas.

**Percentiles:** Los percentiles muestran cómo se distribuyen los datos. Por ejemplo, el 25% de los usuarios han calificado 35 películas o menos. El 50% de los usuarios han calificado 70 películas o menos.

In [131]:
#Se crea un dataframe que contiene solo usuarios cuya cantidad de calificaciones está entre 50 y 1000.

# Consulta SQL para filtrar usuarios con más de 50 calificaciones pero menos de 1000
query = '''
    SELECT userId as user_id,
           COUNT(*) as cnt_rat
    FROM ratings
    GROUP BY userId
    HAVING cnt_rat >= 50 AND cnt_rat <= 1000
    ORDER BY cnt_rat DESC
'''
# Ejecutar la consulta y cargar los resultados en un DataFrame
rating_users2 = pd.read_sql(query, conn)

# Imprimir el DataFrame
print(rating_users2)

     user_id  cnt_rat
0        182      977
1        307      975
2        603      943
3        298      939
4        177      904
..       ...      ...
368      457       50
369      237       50
370      170       50
371      124       50
372       31       50

[373 rows x 2 columns]


Con el nuevo dataframe, se tiene que el usuario con mayor cantidad de calificaciones es el 182 con 977 y el de menor cantidad de calificaciones son varios usuarios, algunos de ellos son: 31, 124, 170, 237, cada unoc on 50 calificaciones.


In [132]:
### Ver distribución después de filtros
rating_users2.describe()

,user_id,cnt_rat
count,373.000000,373.000000
mean,301.766756,201.863271
std,178.462796,187.219913
min,1.000000,50.000000
25%,141.000000,77.000000
50%,305.000000,129.000000
75%,455.000000,248.000000
max,608.000000,977.000000


Hay un total de 373 usuarios que tienen entre 50 y 1000 calificaciones.

El promedio de calificaciones por usuario es aproximadamente 201.86, lo que sugiere que en promedio, los usuarios han calificado alrededor de 201 películas.

La desviación estándar es de aproximadamente 187.22, lo que indica que la dispersión de la cantidad de calificaciones entre los usuarios es considerable.

El usuario que ha aportado menor número de calificaciones, ha aportado 50 calificaciones, mientras que el usuario que ha aportado el mayor número de calificaciones, ha aportado 977. Esto muestra una amplia variabilidad en el número de calificaciones entre los usuarios.

Los cuartiles muestran que el 25% de los usuarios han aportado menos de 77 calificaciones, el 50% han aportado menos de 129 calificaciones y el 75%  menos de 248 calificaciones.

En general, la mayoría de los usuarios han aportado una cantidad considerable de calificaciones, con una variabilidad significativa en la cantidad de calificaciones entre los usuarios.

In [133]:
### graficar distribución después de filtrar datos
fig  = px.histogram(rating_users2, x= 'cnt_rat', title= 'Hist frecuencia de número de calificaciones por usuario')
fig.show()

En el histograma, se evidencia que hay una mayor cantidad de usuarios que han calificado entre 50 y 99 películas; 137 en total. Le siguen 72 usuarios que han calificado entre 100 y 149 películas.

## Distribución de calificaciones por película

In [134]:
# Consulta SQL para calcular la distribución de calificaciones por película
query = """
        SELECT
            movieId,
            COUNT(*) AS cnt_rat
        FROM
            ratings
        GROUP BY
            movieId
        ORDER BY
            cnt_rat DESC
        """

# Ejecutar la consulta y cargar los resultados en un DataFrame
rating_movies = pd.read_sql_query(query, conn)

# Imprimir el DataFrame resultante
print(rating_movies)

      movieId  cnt_rat
0         356      329
1         318      317
2         296      307
3         593      279
4        2571      278
...       ...      ...
9719       96        1
9720       83        1
9721       77        1
9722       55        1
9723       49        1

[9724 rows x 2 columns]


La película con mayor cantidad de calificaciones es la 356, con un total de 329 calificaciones.

In [135]:
### analizar distribución de calificaciones por película
rating_movies.describe()

,movieId,cnt_rat
count,9724.000000,9724.000000
mean,42245.024373,10.369807
std,52191.137320,22.401005
min,1.000000,1.000000
25%,3245.500000,1.000000
50%,7300.000000,3.000000
75%,76739.250000,9.000000
max,193609.000000,329.000000


Cantidad promedio de calificaciones por película: La cantidad promedio de calificaciones por película es de aproximadamente 10.

Número mínimo y máximo de calificaciones por película: El número mínimo de calificaciones por película es 1, lo que indica que hay usuarios que han calificadomuy pocas películas, mientras que el número máximo de calificaciones por película es de 9724, lo que indica que hay usuarios que han calificado un gran número de películas y que el rango es demasiado alto.
La desviación estándar es alta, lo que indica que la cantidad de calificaciones por película está bastante dispersa.

Percentiles: El 25% de las películas han recibido 1 calificación. El 50% de las películas han recibido 3 calificaciones o menos.

In [136]:
### graficar distribución
fig  = px.histogram(rating_movies, x= 'cnt_rat', title= 'Hist frecuencia de número de calificaciones para cada película')
fig.show()

In [137]:
#Se crea un dataframe que contiene solo películas cuya cantidad de calificaciones seaigual o mayor que 50.

###Filtrar películas que tengan 50 o más calificaciones
rating_movies2 = pd.read_sql('''
    SELECT movieId, COUNT(*) AS cnt_rat
    FROM ratings
    GROUP BY movieId
    HAVING cnt_rat >= 50
    ORDER BY cnt_rat DESC
''', conn)

In [138]:
### analizar distribución de calificaciones por película después de filtros
rating_movies2.describe()

,movieId,cnt_rat
count,450.000000,450.000000
mean,11245.017778,91.911111
std,23457.084170,46.098385
min,1.000000,50.000000
25%,757.000000,59.000000
50%,2020.000000,76.000000
75%,4994.500000,107.750000
max,122904.000000,329.000000


Ahora, se tiene un total de 450 películas, cuyo promedio de calificaciones es de casi 92. El número mínimo de calificaciones por el filtro es de 50 y el máximo es de 329.
Ahora el 25% de las películas cuentan con un número de calificaciones de 59 o menos (no menos de 50), el 50% cuentan con 76 calificaciones o menos.

In [141]:
rating_movies2.describe()
fig  = px.histogram(rating_movies2, x= 'cnt_rat', title= 'Hist frecuencia de número de calificaciones para cada película')
fig.show()

# Cerrar la conexión
conn.close()

De acuerdo con el histograma, se tiene que: hay 115 películas que cuentan con un número de calificaciones entre 50 y 59. Seguidas de 76 películas que cuentan con un número de calificaciones que va de 60 a 69. Solo hay una película que tiene un número de calificaciones entre 320 y 329.
